In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd "drive/My Drive/recvis20_a3/"

/content/drive/My Drive/recvis20_a3


In [3]:
%pip install pyyaml==5.1
%pip install imgaug==0.2.5
!git clone 'https://github.com/facebookresearch/detectron2' detectron2

     |████████████████████████████████| 276kB 12.1MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44075 sha256=3400f970e6a89889795b5c043c5fa96fe87e129b3079bb961c2b0b24774b39ee
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 563kB 11.7MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.5-cp36-none-any.whl size=561440 sha256=f7cd75534dc922a3fd47fe80348fcc62d064cff064283b02a5e9dc0c9ff749f3
  Stored in directory: /root/.cache/pip/wheels/31/48/c8/ca3345e8582a078de94243996e148377ef66fdb845557bae0b
Successfully built imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
Cloning into 'detectron2'...
remote: Enumerating objects: 90, done.
remote: 

In [4]:
%pip install -e detectron2

Obtaining file:///content/drive/My%20Drive/recvis20_a3/detectron2
     |████████████████████████████████| 2.2MB 19.4MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.2.post20201111-cp36-none-any.whl size=44418 sha256=987f5590eb397d4a7940f28f81325fbce65d0f6653cf2df1af3795c5938f7756
  Stored in directory: /root/.cache/pip/wheels/70/d9/52/49a17f0d21e723c371c4a3f60138545e0a4163c5f7ff42c855
Successfully built fvcore
  Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0
  Running setup.py develop for detectron2


In [82]:
import os
import sys
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
sys.path.append('detectron2')
import glob
import argparse
import torch

import cv2
import numpy as np
from google.colab.patches import cv2_imshow

from detectron2.utils.logger import setup_logger
#setup_logger()

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# Path and args
path_project = os.getcwd()
path_configs = os.path.join(path_project, 'detectron2', 'configs', 'COCO-InstanceSegmentation')
path_config = os.path.join(path_configs, 'mask_rcnn_R_101_FPN_3x.yaml')

In [119]:
# Config for mask-rcnn
cfg = get_cfg()
cfg.merge_from_file(path_config)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8  # Bounding box threshold confidence
cfg.MODEL.ROI_HEADS.NMS = 0.5  # Non Maximum Suppression threshold
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")  # pretrained on Imagenet
model = DefaultPredictor(cfg)

In [120]:
def Detector(model, dataset, cat, save=True):
  '''
  model: mask-rcnn
  dataset: 'train', 'val' or 'test'
  cat: bird category
  save: Boolean => save the bounding boxes or not

  Detector predicts the BB of category 'cat' of dataset 'dataset' and returns 
  the number of images without bird prediction
  '''
  path_imgs = glob.glob('bird_dataset/{}_images/{}/*.jpg'.format(dataset, cat))

  #Path_save
  os.makedirs('cropped_birds', exist_ok=True)
  os.makedirs('cropped_birds/{}_cropped'.format(dataset), exist_ok=True)

  base_save_path = 'cropped_birds/{}_cropped/{}'.format(dataset, cat)
  if save:
    os.makedirs(base_save_path, exist_ok=False)

  no_box = 0
  for img in tqdm(path_imgs):

    im = cv2.imread(img)
    output = model(im)
    index = np.where(output['instances'].pred_classes.cpu().numpy() == 14)[0]  # 14 is bird class

    if len(index) == 1:

      #Path_save
      im_name = os.path.basename(img)
      path_save = os.path.join(base_save_path, im_name)

      y0, x0, y1, x1 = list(output['instances'].pred_boxes)[0].cpu().numpy()
      cropped_image = im[int(x0):int(x1), int(y0): int(y1), ::-1]
      cropped_img = Image.fromarray(cropped_image)

      if save:
        cropped_img.save(path_save)

    elif len(index) > 1:

      for i in index:

        #Path_save
        im_name = os.path.basename(img).split(r'.')
        path_save = os.path.join(base_save_path, im_name[0] + '_{}.jpg'.format(i))

        y0, x0, y1, x1 = list(output['instances'].pred_boxes)[i].cpu().numpy()
        cropped_image = im[int(x0):int(x1), int(y0): int(y1), ::-1]
        cropped_img = Image.fromarray(cropped_image)

        if save:
          cropped_img.save(path_save)

    else:

      no_box += 1

  return no_box

In [122]:
categories = [cat for cat in os.listdir('bird_dataset/train_images') if cat != '.DS_Store']
test_cat = 'mistery_category'

miss_train = []
miss_dev = []
for cat in categories:
  print('\n' + cat)
  no_train = Detector(model, 'train', cat, save=True)
  no_dev = Detector(model, 'val', cat, save=True)
  miss_train.append(no_train)
  miss_dev.append(no_dev)

print('\nTest')
miss_test = Detector(model, 'test', 'mistery_category', save=True)

print('\n' + 'No bird detected in {} Train images'.format(np.sum(miss_train)))
print('No bird detected in {} Dev images'.format(np.sum(miss_dev)))
print('No bird detected in {} Test images'.format(miss_test))

  0%|          | 0/51 [00:00<?, ?it/s]

030.Fish_Crow


  2%|▏         | 1/48 [00:00<00:08,  5.33it/s]

012.Yellow_headed_Blackbird


  0%|          | 0/50 [00:00<?, ?it/s]

...

010.Red_winged_Blackbird


  0%|          | 0/517 [00:00<?, ?it/s]

Test


100%|██████████| 517/517 [01:40<00:00,  5.14it/s]


No bird detected in 12 Train images
No bird detected in 5 Dev images
No bird detected in 16 Test images
